# Customer Segmentation - Step 1: Data Preparation

In [1]:
# a dataset named 'customer_segmentation' will be created under the default project
output_dataset = 'USER_SXM4690.'

cs_id_table = output_dataset + 'cs_id'
cs_seed_table = output_dataset + 'cs_seed'
cs_filter_table = output_dataset + 'cs_filter'

In [2]:
%%bq query -n cs_id

select
    a.cust_id,
    a.s_trd_typ_desc,
    'PAR' as ass_chan
from
`pr-customer-thd.EDW_REWARDS.PRO_CUST_DTL` a
where 1 = 1
and a.trtry_typ_cd = 1

union all

select
    a.cust_id,
    a.s_trd_typ_desc,
    'PASA' as ass_chan
from
`pr-customer-thd.EDW_REWARDS.PRO_CUST_DTL` a,
unnest(pasa_dtl) b
where 1 = 1
and b.assg_pasa_eff_bgn_ts is not null 
and b.assg_pasa_eff_end_ts is null 
and (
a.trtry_typ_cd in (2,3)
or
a.trtry_typ_cd is null
)


In [3]:
%%bq execute -q cs_id -t $cs_id_table -m overwrite

cust_id,s_trd_typ_desc,ass_chan
79810,Other,PAR
1550205,Other,PAR
769738,Other,PAR
2281320,Other,PAR
3993671,Other,PAR
865836,Other,PAR
1833424,Other,PAR
3995349,Other,PAR
813903,Other,PAR
775513,Other,PAR


In [4]:
%%bq query -n cs_seed
select
    a.cust_id,
    a.s_trd_typ_desc,
    c.dept_nbr,
    c.dept_nm,
    c.item_class_cd,
    c.item_class_desc,
    c.categ_r12_sls_amt,
    d.tot_r12_txn_cnt,
    d.tot_r12_sls_amt
from
USER_SXM4690.cs_id a
left join
(
    select
        a.cust_id,
        c.CLASS_CTG_GRP_CD as dept_nbr,
        c.ITEM_GRP_DESC as dept_nm,
        c.item_class_cd,
        c.item_class_desc,
        sum(coalesce(b.net_sls_amt,0)) as categ_r12_sls_amt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
        unnest(hdr_dtl) b,
        unnest(item_dtl) c,
        `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` d
          where 1 = 1
          and a.sltrn_dt = d.cal_dt
          and a.cust_id <> 0
          and a.cust_id is not null
          and c.ITEM_CORE_DEPT_DESC = 'Core Department'
          and c.CLASS_CTG_GRP_CD not in ('0025', '0027', '0028')
          and c.CLASS_CTG_GRP_CD is not null
          and c.item_class_cd is not null
          and b.net_sls_amt > 0
          and d.cal_dt between
            (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
            and
            (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)
      group by 1,2,3,4,5
) c
on a.cust_id = c.cust_id
left join
(
    select
        a.cust_id,
        count(distinct concat(cast(a.loc_nbr as string),' ',cast(a.sltrn_dt as string),' ',cast(a.pos_rgstr_id as string),' ',cast(a.sltrn_id as string))) as tot_r12_txn_cnt,
        sum(coalesce(b.net_sls_amt,0)) as tot_r12_sls_amt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
        unnest(hdr_dtl) b,
        unnest(item_dtl) c,
        `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` d
          where 1 = 1
          and a.sltrn_dt = d.cal_dt
          and a.cust_id <> 0
          and a.cust_id is not null
          and c.ITEM_CORE_DEPT_DESC = 'Core Department'
          and c.CLASS_CTG_GRP_CD not in ('0025', '0027', '0028')
          and c.CLASS_CTG_GRP_CD is not null
          and c.item_class_cd is not null
          and b.net_sls_amt > 0
          and d.cal_dt between
            (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
            and
            (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)
      group by 1
) d
on a.cust_id = d.cust_id
where 1 = 1

In [5]:
%%bq execute -q cs_seed -t $cs_seed_table -m overwrite

cust_id,s_trd_typ_desc,dept_nbr,dept_nm,item_class_cd,item_class_desc,categ_r12_sls_amt,tot_r12_txn_cnt,tot_r12_sls_amt
5232854,,028I,GRD/INDOOR,21,STORAGE BUILDINGS,387.99,121,14535.3
37918029,TV/Movie Productions,028I,GRD/INDOOR,26,PATIO FURNITURE,99.0,447,62039.22
4640924,Exporter,029B,KITCHEN AND BATH,21,KITCHEN SINKS,105.93,157,53351.56
3307858,Exporter,028I,GRD/INDOOR,26,PATIO FURNITURE,2253.25,15,13229.61
828325,Exporter,029A,APPLIANCES,61,APPLIANCE INSTALLATION,55.0,7,78956.29
340867,Exporter,029A,APPLIANCES,18,DELIVERY,59.0,832,118956.39
1664177,Housing Authority,029A,APPLIANCES,17,A/CS AND FANS,1098.0,85,69195.03
4707547,Housing Authority,029A,APPLIANCES,7,DISHWASHERS,227.0,185,44797.77
819533,Housing Authority,0022,BUILDING MATERIALS,90,MISCELLANEOUS,75.0,265,362596.91
80509,Housing Authority,029A,APPLIANCES,16,FLOORCARE,33.94,886,227557.75


In [6]:
%%bq query -n cs_filter
select
    a.cust_id,
    a.s_trd_typ_desc,
    a.ass_chan,
    a.pro_xtra_tenure,
    coalesce(a.mgmt_tenure,'NEVER MANAGED') as mgmt_tenure,
    coalesce(a.dual_mgmt,'N') as dual_mgmt,
    coalesce(a.pasa_algnmnt,'NOT PASA-ASSIGNED') as pasa_algnmnt,
    a.par_algnmnt,
    coalesce(a.pyrmd_tier,'NO SALES') as pyrmd_tier,
    coalesce(a.top_mkt_nm,'NO SALES') as top_mkt_nm,
    case when a.top_mkt_perc > 100
         then 'N/A'
         when a.top_mkt_perc > .9 and a.top_mkt_perc <= 100
         then 'A: > 90%'
         when a.top_mkt_perc > .8 and a.top_mkt_perc <= .9
         then 'B: > 80%'
         when a.top_mkt_perc > .7 and a.top_mkt_perc <= .8
         then 'C: > 70%'
         when a.top_mkt_perc > .6 and a.top_mkt_perc <= .7
         then 'D: > 60%'
         when a.top_mkt_perc > .5 and a.top_mkt_perc <= .6
         then 'E: > 50%'
         when a.top_mkt_perc <= .5
         then 'F: <= 50%'
         end as top_mkt_rng,
    case when a.vly_perc > 100
         then 'N/A'
         when a.vly_perc > .15 and a.vly_perc <= 100
         then 'A: > 15%'
         when a.vly_perc > .05 and a.vly_perc <= .15
         then 'B: BETWEEN 5% AND 15%'
         when a.vly_perc > -.05 and a.vly_perc <= .05
         then 'C: BETWEEN -5% AND 5%'
         when a.vly_perc > -.15 and a.vly_perc <= -.05
         then 'D: BETWEEN -15% AND -5%'
         when a.vly_perc <= -.15
         then 'E: <= -15%'
         end as vly_rng,
    a.vpp_plus_bulk_discount as vpp_plus_bulk_rng,
    a.gm_rate as gm_rate_rng
  from
(
select 
    a.cust_id,
    a.s_trd_typ_desc,
    a.ass_chan,
    t.pro_xtra_tenure,
    coalesce(t.mgmt_tenure,'NEVER MANAGED') as mgmt_tenure,
    b.dual_mgmt,
    c.pasa_algnmnt,
    d.par_algnmnt,
    case when e.tot_r12_sls_amt > 100000
         then 'A: > 100K'
         when e.tot_r12_sls_amt > 35000 and e.tot_r12_sls_amt <= 100000
         then 'B: > 35K'
         when e.tot_r12_sls_amt > 0 and e.tot_r12_sls_amt <= 35000
         then 'C: > 0K'
         when coalesce(e.tot_r12_sls_amt,0) <= 0
         then 'D: <= 0K'
         end as pyrmd_tier,
    g.top_mkt_nm,
    case when h.vpp_plus_bulk_discount > .25
         then 'A: > 25%'
         when h.vpp_plus_bulk_discount > .15 and h.vpp_plus_bulk_discount <= .25
         then 'B: > 15%'
         when h.vpp_plus_bulk_discount > .05 and h.vpp_plus_bulk_discount <= .15
         then 'C: > 5%'
         when h.vpp_plus_bulk_discount >= 0 and h.vpp_plus_bulk_discount <= .05
         then 'D: >= 0%'
         when coalesce(h.vpp_plus_bulk_discount,0) < 0
         then 'E: N/A'
         end as vpp_plus_bulk_discount,
    case when h.gm_rate > .45
         then 'A: > 45%'
         when h.gm_rate > .30 and h.gm_rate <= .45
         then 'B: > 30%'
         when h.gm_rate > .15 and h.gm_rate <= .30
         then 'C: > 15%'
         when h.gm_rate > 0 and h.gm_rate <= .15
         then 'D: > 0%'
         when coalesce(h.gm_rate,0) <= 0
         then 'E: SALES BELOW COST'
         end as gm_rate,
    sum(if(coalesce(e.tot_r12_sls_amt,0) = 0,101,(g.top_mkt_sls_amt/e.tot_r12_sls_amt))) as top_mkt_perc,
    sum(if(coalesce(f.tot_ly_r12_sls_amt,0) = 0, 101,(e.tot_r12_sls_amt/f.tot_ly_r12_sls_amt)-1)) as vly_perc

from USER_SXM4690.cs_id a

left join
(
    select
        a.cust_id,
        'Y' as dual_mgmt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_DTL` a,
        unnest(pasa_dtl) b
          where 1 = 1
          and b.assg_pasa_eff_bgn_ts is not null 
          and b.assg_pasa_eff_end_ts is null 
          and a.trtry_typ_cd = 1
) b
on a.cust_id = b.cust_id
left join
(
    select distinct
            a.cust_id,
            b.mkt_nm as pasa_algnmnt
          from
           `pr-customer-thd.DB2_REWARDS.CASSOC_REL` a,
           `pr-finance-thd.FINANCE_DATA.STR_HIER` b
              where 1 = 1
              and a.str_nbr = b.str_nbr
              and a.eff_end_ts is null
) c
on a.cust_id = c.cust_id
left join
(
    select
        a.cust_id,
        a.s_trd_typ_desc,
        if(a.trtry_typ_cd = 1, concat(a.div_nm,', ',a.orgn_nm),'NOT PAR-MANAGED') as par_algnmnt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_DTL` a
          where 1 = 1
) d
on a.cust_id = d.cust_id
left join
(
    select
        a.cust_id,
        count(distinct concat(cast(a.loc_nbr as string),' ',cast(a.sltrn_dt as string),' ',cast(a.pos_rgstr_id as string),' ',cast(a.sltrn_id as string))) as tot_r12_txn_cnt,
        sum(coalesce(b.net_sls_amt,0)) as tot_r12_sls_amt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
        unnest(hdr_dtl) b,
        unnest(item_dtl) c,
        `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` d
          where 1 = 1
          and a.sltrn_dt = d.cal_dt
          and a.cust_id <> 0
          and a.cust_id is not null
          and c.ITEM_CORE_DEPT_DESC = 'Core Department'
          and b.net_sls_amt > 0
          and d.fscl_prd_key_val between
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
                (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)))
            and
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
      group by 1
) e
on a.cust_id = e.cust_id
left join
(
    select
        a.cust_id,
        sum(coalesce(b.net_sls_amt,0)) as tot_ly_r12_sls_amt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
        unnest(hdr_dtl) b,
        unnest(item_dtl) c,
        `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` d
          where 1 = 1
          and a.sltrn_dt = d.cal_dt
          and a.cust_id <> 0
          and a.cust_id is not null
          and c.ITEM_CORE_DEPT_DESC = 'Core Department'
          and b.net_sls_amt > 0
          and d.fscl_prd_key_val between
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
                (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = 
                  (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))))
            and
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
                (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
                  (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))))
      group by 1
) f
on a.cust_id = f.cust_id
left join
(
    select
        a.cust_id,
        a.top_mkt_nm,
        a.top_mkt_sls_amt
      from
    (
    select
        a.cust_id,
        d.mkt_nm as top_mkt_nm,
        rank() over(partition by a.cust_id order by sum(coalesce(b.net_sls_amt,0)) desc) as rank,
        sum(coalesce(b.net_sls_amt,0)) as top_mkt_sls_amt
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
        unnest(hdr_dtl) b,
        unnest(item_dtl) c,
        unnest(loc_dtl) d,
        `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` e
          where 1 = 1
          and a.sltrn_dt = e.cal_dt
          and a.cust_id <> 0
          and a.cust_id is not null
          and c.ITEM_CORE_DEPT_DESC = 'Core Department'
          and b.net_sls_amt > 0
          and e.fscl_prd_key_val between
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
                (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)))
            and
            (select fscl_prd_key_val from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
              (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
      group by 1,2
    ) a
          where 1 = 1
          and a.rank = 1
) g
on a.cust_id = g.cust_id
left join
(
select
    a.cust_id,
    case when a.tot_sls_amt = 0
      then 0
      else (a.tot_mrgn_amt)/(a.tot_sls_amt) 
      end as gm_rate,
    case when a.tot_sls_amt = 0
      then 0
      else (a.unq_comp + a.bulk)/(a.tot_sls_amt + a.bulk + a.unq_comp + 0.000001) 
      end as vpp_plus_bulk_discount
  from
(
select
    a.cust_id,
    sum(coalesce(a.tot_mrgn_amt,0)) as tot_mrgn_amt,
    sum(coalesce(a.tot_sls_amt,0)) as tot_sls_amt,
    sum(coalesce(a.unq_comp,0)) as unq_comp, -- unique competitive (vpp), buyer mkdn
    sum(coalesce(a.bulk,0)) as bulk -- bulk, buyer mkdn,
  from
(
select
    a.cust_id,
    a.txn_id,
    a.item_sku_nbr,
    sum(coalesce(a.tot_mrgn_amt,0)) as tot_mrgn_amt,
    sum(coalesce(a.tot_sls_amt,0)) as tot_sls_amt,
    sum(coalesce(b.unq_comp,0)) as unq_comp, -- unique competitive (vpp), buyer mkdn
    sum(coalesce(b.bulk,0)) as bulk -- bulk, buyer mkdn
  from
(
select
    a.cust_id,
    concat(cast(a.loc_nbr as string),' ',cast(a.sltrn_dt as string),' ',cast(a.pos_rgstr_id as string),' ',cast(a.sltrn_id as string)) as txn_id,
    b.item_sku_nbr,
    sum(coalesce(b.gross_mrgn_amt,0)) as tot_mrgn_amt, 
    sum(coalesce(b.net_sls_amt,0)) as tot_sls_amt
  from
    `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
    unnest(hdr_dtl) b,
    unnest(item_dtl) c,
    `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` d
      where 1 = 1
      and a.sltrn_dt = d.cal_dt
      and a.cust_id <> 0
      and a.cust_id is not null
      and c.ITEM_CORE_DEPT_DESC = 'Core Department'
      and b.net_sls_amt > 0
      and d.cal_dt between
        (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
          (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
        and
        (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)
  group by 1,2,3
) a
left join
(
select
    concat(cast(a.loc_nbr as string),' ',cast(a.sltrn_dt as string),' ',cast(a.pos_rgstr_id as string),' ',cast(a.sltrn_id as string)) as txn_id,
    b.item_sku_nbr,
    sum(coalesce(b.rsn30_ucs_mumd_amt,0)) as unq_comp, -- unique competitive (vpp), buyer mkdn
    sum(coalesce(b.rsn49_qty_prc_mumd_amt,0))as bulk -- bulk, buyer mkdn
  from
    `pr-customer-thd.EDW_REWARDS.PRO_CUST_SLS_DTL` a,
    unnest(mumd_dtl) b, 
    `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` c
      where 1 = 1
      and a.sltrn_dt = c.cal_dt
      and c.cal_dt between
        (select fscl_prd_bgn_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt =
          (select (ly_fscl_wk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date))
        and
        (select prev_fprd_end_dt from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date)
  group by 1,2
) b
on a.txn_id = b.txn_id
and a.item_sku_nbr = b.item_sku_nbr
      where 1 = 1 
  group by 1,2,3
) a
  group by 1
) a
      where 1 = 1
) h
on a.cust_id = h.cust_id
left join
(
    select
        a.cust_id,
        a.s_trd_typ_desc,
        case when cast(round(date_diff((select (prev_fwk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date),
               a.proxtr_mbrshp_bgn_dt,day)/7) as int64) > 260
               then '+5 YEARS'
             when cast(round(date_diff((select (prev_fwk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date),
               a.proxtr_mbrshp_bgn_dt,day)/7) as int64) between 208 and 260
               then '4-5 YEARS'
             when cast(round(date_diff((select (prev_fwk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date),
               a.proxtr_mbrshp_bgn_dt,day)/7) as int64) between 156 and 208
               then '3-4 YEARS'
             when cast(round(date_diff((select (prev_fwk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date),
               a.proxtr_mbrshp_bgn_dt,day)/7) as int64) between 104 and 156
               then '2-3 YEARS'
             when cast(round(date_diff((select (prev_fwk_end_dt) from `pr-finance-thd.FINANCE_DATA.CAL_PRD_HIER` where cal_dt = current_date),
               a.proxtr_mbrshp_bgn_dt,day)/7) as int64) between 52 and 104
               then '1-2 YEARS'
             else '0-1 YEARS' 
             end as pro_xtra_tenure,
        case when a.fxd_wks_mgd_cnt >= 1 and a.fxd_wks_mgd_cnt <=52
          then '0-1 YEARS'
          when a.fxd_wks_mgd_cnt > 52 and a.fxd_wks_mgd_cnt <= 104
          then '1-2 YEARS'
          when a.fxd_wks_mgd_cnt > 104 and a.fxd_wks_mgd_cnt <= 156
          then '2-3 YEARS'
          when a.fxd_wks_mgd_cnt > 156
          then '3+ YEARS'
          end as mgmt_tenure 
      from
        `pr-customer-thd.EDW_REWARDS.PRO_CUST_DTL` a
          where 1 = 1
)  t
on a.cust_id = t.cust_id
where 1 = 1
group by 1,2,3,4,5,6,7,8,9,10,11,12
) a
where 1 = 1 
order by 1


In [7]:
%%bq execute -q cs_filter -t $cs_filter_table -m overwrite

cust_id,s_trd_typ_desc,ass_chan,pro_xtra_tenure,mgmt_tenure,dual_mgmt,pasa_algnmnt,par_algnmnt,pyrmd_tier,top_mkt_nm,top_mkt_rng,vly_rng,vpp_plus_bulk_rng,gm_rate_rng
24,Property Manager,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,B: > 35K,DLS/FT WTH,A: > 90%,D: BETWEEN -15% AND -5%,D: >= 0%,B: > 30%
36,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,B: > 35K,DLS/FT WTH,A: > 90%,A: > 15%,D: >= 0%,B: > 30%
37,Commercial Constr.,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,A: > 100K,DLS/FT WTH,A: > 90%,A: > 15%,D: >= 0%,B: > 30%
89,Remodeler,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,B: > 35K,DLS/FT WTH,A: > 90%,A: > 15%,D: >= 0%,B: > 30%
100,Commercial Constr.,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,B: > 35K,DLS/FT WTH,A: > 90%,C: BETWEEN -5% AND 5%,C: > 5%,C: > 15%
186,Commercial Constr.,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,B: > 35K,DLS/FT WTH,A: > 90%,D: BETWEEN -15% AND -5%,D: >= 0%,B: > 30%
243,Electrician,PASA,4-5 YEARS,NEVER MANAGED,N,DALLAS/FT WORTH,NOT PAR-MANAGED,C: > 0K,DLS/FT WTH,A: > 90%,A: > 15%,D: >= 0%,A: > 45%
250,Commercial Constr.,PAR,4-5 YEARS,3+ YEARS,N,NOT PASA-ASSIGNED,"SOUTHERN, GULF",A: > 100K,HOUSTON,F: <= 50%,A: > 15%,D: >= 0%,A: > 45%
266,Property Manager,PASA,4-5 YEARS,3+ YEARS,N,DALLAS/FT WORTH,NOT PAR-MANAGED,A: > 100K,DLS/FT WTH,A: > 90%,C: BETWEEN -5% AND 5%,D: >= 0%,B: > 30%
268,Remodeler,PAR,4-5 YEARS,3+ YEARS,N,NOT PASA-ASSIGNED,"SOUTHERN, SOUTHWEST",A: > 100K,DLS/FT WTH,A: > 90%,E: <= -15%,D: >= 0%,C: > 15%
